In [1]:
# Lasciate questa cella così ogni volta che aggiornate un file python, 
# anche gli import nel notebook vengono aggiornati

%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

from Utils import *
from constants import *

from Environment import *
from Non_Stationary_Environment import *

from Social_influence import *
from Network import Network

from Simulation import *

from Ecommerce import *
from Ecommerce3 import *
from Ecommerce4 import *
from Ecommerce5 import *
from Ecommerce6 import *
from step7.Ecommerce7 import*

In [4]:

graph_weights, alpha_bars, product_prices, users_reservation_prices, \
observations_probabilities, users_poisson_parameters = setup_environment()

env = Environment(users_reservation_prices, graph_weights, alpha_bars)

ecomm = Ecommerce(B_cap, budgets, product_prices)
ecomm3_gpts = Ecommerce3_GPTS(B_cap, budgets, product_prices)
ecomm3_gpucb = Ecommerce3_GPUCB(B_cap, budgets, product_prices)


# Every day a new montecarlo simulation must be run to sample num of items sold
num_sold_items = estimate_nodes_activation_probabilities(
    env.network.get_adjacency_matrix(),
    env.users_reservation_prices,
    users_poisson_parameters,
    product_prices,
    observations_probabilities
)

expected_reward = env.compute_clairvoyant_reward(
    num_sold_items,
    product_prices,
    budgets
)     


optimal_allocation , optimal_gain, optimal_allocation_idxs = ecomm.clairvoyant_optimization_problem(expected_reward)
alpha, reward = env.round_step3(pulled_arm = np.array(optimal_allocation), pulled_arm_idxs = np.array(optimal_allocation_idxs))
try:
    assert(optimal_gain == reward)
except AssertionError as ae:
    print(f'optimal_gain: {optimal_gain}, env_reward: {reward}')




AttributeError: 'list' object has no attribute 'shape'

## Step 3

In [ ]:
gpts_rewards_per_experiment, gpucb_rewards_per_experiment, opts = simulate_step3()
plot_regrets(gpts_rewards_per_experiment, gpucb_rewards_per_experiment, opts, ["GPTS", "GPUCB"])

## Step 4

In [ ]:
gpts_rewards_per_experiment, gpucb_rewards_per_experiment, opts = simulate_step4()
plot_regrets(gpts_rewards_per_experiment, gpucb_rewards_per_experiment, opts, ["GPTS", "GPUCB"])

## Step 5

In [ ]:
gpts_rewards_per_experiment, gpucb_rewards_per_experiment, opts = simulate_step5()
plot_regrets(gpts_rewards_per_experiment, gpucb_rewards_per_experiment, opts, ["GPTS", "GPUCB"])

## Step 6

In [ ]:
swucb_rewards_per_experiment, cducb_rewards_per_experiment, opts = simulate_step6()
plot_regrets(swucb_rewards_per_experiment, cducb_rewards_per_experiment, opts, ["SWUCB", "CDUCB"])

## Step 7

In [ ]:
gpts_rewards_per_experiment, gpucb_rewards_per_experiment, opts = simulate_step7()
plot_regrets(gpts_rewards_per_experiment, gpucb_rewards_per_experiment, opts, ["GPTS", "GPUCB"])